**Moooooooodel**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

print(tf.__version__)
print(tf.config.list_physical_devices("GPU"))

Mounted at /content/drive
2.19.0
[]


In [2]:
TRAIN_DIR = "/content/drive/MyDrive/AI_Project/experiment/split/train"
VAL_DIR   = "/content/drive/MyDrive/AI_Project/experiment/split/val"
TEST_DIR  = "/content/drive/MyDrive/AI_Project/experiment/split/test"

IMG_SIZE = (224,224)
BATCH_SIZE = 32

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=42
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=42
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print(NUM_CLASSES)

Found 2853 files belonging to 23 classes.
Found 399 files belonging to 23 classes.
Found 820 files belonging to 23 classes.
23


In [4]:
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=IMG_SIZE + (3,)
)

base_model.trainable = False

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 23)             │         2,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,424,919 (9.25 MB)

 Trainable params: 166,935 (652.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
EPOCHS = 18
best_val_acc = 0.0

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(1e-4)
for epoch in range(EPOCHS):

    # -------- TRAIN --------
    train_correct = 0
    train_total = 0
    train_loss = 0.0

    for images, labels in train_ds:
        with tf.GradientTape() as tape:
            outputs = model(images, training=True)
            loss = loss_fn(labels, outputs)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        preds = tf.argmax(outputs, axis=1, output_type=tf.int32)
        train_correct += tf.reduce_sum(tf.cast(preds == labels, tf.int32))
        train_total += labels.shape[0]
        train_loss += loss.numpy() * labels.shape[0]

    train_acc = train_correct / train_total
    train_loss /= train_total

    # -------- VALIDATION --------
    val_correct = 0
    val_total = 0
    val_loss = 0.0

    for images, labels in val_ds:
        outputs = model(images, training=False)
        loss = loss_fn(labels, outputs)

        preds = tf.argmax(outputs, axis=1, output_type=tf.int32)
        val_correct += tf.reduce_sum(tf.cast(preds == labels, tf.int32))
        val_total += labels.shape[0]
        val_loss += loss.numpy() * labels.shape[0]

    val_acc = val_correct / val_total
    val_loss /= val_total

    print(
        f"Epoch [{epoch+1}/{EPOCHS}] "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model.save("mobilenet_best.h5")


Epoch [1/18] Train Acc: 0.1181 | Val Acc: 0.2256


Epoch [2/18] Train Acc: 0.2622 | Val Acc: 0.3258


Epoch [3/18] Train Acc: 0.4010 | Val Acc: 0.4361


Epoch [4/18] Train Acc: 0.4939 | Val Acc: 0.4912


Epoch [5/18] Train Acc: 0.5633 | Val Acc: 0.5464


...

Epoch [12/18] Train Acc: 0.7764 | Val Acc: 0.6441


Epoch [13/18] Train Acc: 0.7904 | Val Acc: 0.6566


Epoch [14/18] Train Acc: 0.8093 | Val Acc: 0.6642
Epoch [15/18] Train Acc: 0.8198 | Val Acc: 0.6566
Epoch [16/18] Train Acc: 0.8304 | Val Acc: 0.6642


Epoch [17/18] Train Acc: 0.8454 | Val Acc: 0.6717


Epoch [18/18] Train Acc: 0.8521 | Val Acc: 0.6767


In [6]:
test_correct = 0
test_total = 0
test_loss = 0.0

for images, labels in test_ds:
    outputs = model(images, training=False)
    loss = loss_fn(labels, outputs)

    preds = tf.argmax(outputs, axis=1, output_type=tf.int32)
    test_correct += tf.reduce_sum(tf.cast(preds == labels, tf.int32))
    test_total += labels.shape[0]
    test_loss += loss.numpy() * labels.shape[0]

test_acc = test_correct / test_total
test_loss /= test_total

print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

Test Accuracy: 0.6561
Test Loss: 1.2086


In [7]:
print(f"Baseline Test Accuracy: {test_acc:.4f}")

Baseline Test Accuracy: 0.6561


In [11]:
# ===== Fine-Tuning Setup (صح) =====

base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)


In [12]:
EPOCHS_FINE = 12

for epoch in range(EPOCHS_FINE):
    print(f"Fine-tune Epoch {epoch+1}")

    for images, labels in train_ds:
        with tf.GradientTape() as tape:
            outputs = model(images, training=True)
            loss = loss_fn(labels, outputs)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

Fine-tune Epoch 1
Fine-tune Epoch 2
Fine-tune Epoch 3
Fine-tune Epoch 4
Fine-tune Epoch 5
Fine-tune Epoch 6
Fine-tune Epoch 7
Fine-tune Epoch 8
Fine-tune Epoch 9
Fine-tune Epoch 10
Fine-tune Epoch 11
Fine-tune Epoch 12


In [13]:
test_correct = 0
test_total = 0

for images, labels in test_ds:
    outputs = model(images, training=False)
    preds = tf.argmax(outputs, axis=1, output_type=tf.int32)

    test_correct += tf.reduce_sum(tf.cast(preds == labels, tf.int32))
    test_total += labels.shape[0]

test_acc = test_correct / test_total
print(f"Test Accuracy after fine-tuning: {test_acc:.4f}")

Test Accuracy after fine-tuning: 0.7049


In [14]:
model.save("mobilenet_finetuned_best.h5")
print("✅ Final model saved")

✅ Final model saved
